# Lectura y preprocesamiento de documentos

In [1]:
# Paquetes necesarios

import re
import pandas as pd
import os
import PyPDF2
import time
import sys
import pickle

# Packages for text preprocessing
import nltk

## Almacenar los documentos en una base de datos

In [2]:
#---------------------------------------
# Cargar cada pdf en una lista de Python
#---------------------------------------

#scriptPath = 'C:\\Users\\User\\Desktop\\DS4A_workspace\\Final Project\\DS4A-Final_Project-Team_28\\Personal\\Camilo'

# Directorio de trabajo absoluto
# Necesario para llamados relativos desde bucles
scriptPath = sys.path[0]
os.chdir(scriptPath + '/Last_conpes_pdfs')

# Directorio con los pdfs
files = os.listdir(scriptPath + '/Last_conpes_pdfs')


# Lista vacía para llenar con cada pdf almacenado
Database = []

# Bucle que abre cada pdf en Python
for FILE in files:
    # Si termina en '.pdf' leerlo en formato binario y pegarlo a la lista
    if FILE.endswith('.pdf'):
        data = open(FILE,'rb') 
        Database.append(data)

Observemos los primeros 10 archivos

In [3]:
Database[0:10]

[<_io.BufferedReader name='3873.pdf'>,
 <_io.BufferedReader name='3874.pdf'>,
 <_io.BufferedReader name='3875.pdf'>,
 <_io.BufferedReader name='3876.pdf'>,
 <_io.BufferedReader name='3877.pdf'>,
 <_io.BufferedReader name='3878.pdf'>,
 <_io.BufferedReader name='3879.pdf'>,
 <_io.BufferedReader name='3880.pdf'>,
 <_io.BufferedReader name='3881.pdf'>,
 <_io.BufferedReader name='3882.pdf'>]

## Leer los documentos con bucle

Con el paquete PyPDF2 podemos leer archivos en pdf con facilidad. Sin embargo, hay varios documentos que no se encuentran en formato legible, es decir que para ellos hay que emplear el procedimiento de Reconocimiento Óptico de Caracteres. En este caso obviaremos dichos documentos por ser minoría.

In [4]:
# Dataframe vacío que almacenará el nombre del PDF y su texto
text_table = pd.DataFrame(index = [0], columns = ['PDF','Text'])

fileIndex = 0

# Cntabilizar tiempo de ejecución
t0 = time.time()

# Para cada pdf que puede ser leído
for file in files:
  # Leer como binario
  pdfFileObj = open(file,'rb')
  # Magia con PyPDF2
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  # Empezar contador de páginas
  startPage = 0
  # Caracter vacío para rellenarse con texto
  text = ''
  cleanText = ''
  # Mientras que el contador sea menor al número de páginas
  while startPage <= pdfReader.numPages-1:
    pageObj = pdfReader.getPage(startPage)
    text += pageObj.extractText()
    startPage += 1
  pdfFileObj.close()
  # Para cada palabra dentro del texto
  for myWord in text:
    # Ignorar saltos de línea
    if myWord != '\n':
      cleanText += myWord
  # Objeto temporal que contiene el texto limpio
  text = cleanText
  # Crear fila vacía
  newRow = pd.DataFrame(index = [0], columns = ['PDF', 'Text'])
  # Rellenar la anterior fila con nombre y texto
  newRow.iloc[0]['PDF'] = file
  newRow.iloc[0]['Text'] = text
  # Concatenar la fila creada al dataframe creado por fuera del bucle
  text_table = pd.concat([text_table, newRow], ignore_index=True)        


t1 = time.time()

# Tiempo total de ejecución
total = t1-t0

Observemos el dataframe creado

In [5]:
text_table = text_table.iloc[1:]
text_table

,PDF,Text
1,3873.pdf,Documento CONPES CONSEJO NACIO...
2,3874.pdf,Documento CONPES CONSEJO NACION...
3,3875.pdf,Documento CONPES CONSEJO NACIO...
4,3876.pdf,Documento CONPES CONSEJO NACI...
5,3877.pdf,Documento CONPES CONSEJO NACIO...
...,...,...
89,3965.pdf,1 Documento CONPES CONSEJO NACIO...
90,3966.pdf,Documento CONPES CONSEJO NACI...
91,3968.pdf,Documento CONPES CONSEJO NACION...
92,3969.pdf,Documento CONPES CONSEJO NACI...


## Limpieza del texto

En procesamiento de lenguaje natural, para que una computadora pueda interpretar las palabras como números, es necesario realizar transformaciones sobre estas. En ese sentido, el texto se homogeniza retirando todos los elementos que no le aportan al verdadero significado del texto. Entre esos elementos encontramos:

- Caracteres especiales.
- Palabras mal escritas de corta longitud.
- Acentos si estos con inconsistentes (en español corregir la ortografía es más complicado que en inglés).
- Stop words: artículos, conectores y palabras que dependiendo del contexto se consideren stop words.

In [6]:
# Remover caracteres que no son ASCII
def remove_noChar(words):
    return [re.sub(u"[^a-zA-ZñÑáéíóúÁÉÍÓÚ ]","", word) for word in words]

# Remover stopwords
def remove_sw(words,sw_list):
    return [word for word in words if word not in sw_list]

# Remover caracteres cortos
def remove_shortW(words):
    return [word for word in words if len(word) > 2]

# Remover acentos
# Esto no es necesario para documentos que tienen buena ortografía
def remove_tilde(words):
    return [r_tilde(word) for word in words]

# Reemplazo de tildes
def r_tilde(word):
    w=[]
    for letra in list(word):
        if letra == 'á': letra = 'a'
        if letra == 'é': letra = 'e'
        if letra == 'í': letra = 'i'
        if letra == 'ó': letra = 'o'
        if letra == 'ú': letra = 'u'
        w += letra
    return ''.join(w)

# Función que usa las funciones anteriores
def preProc_docs(documentos):
    documentos = [re.sub(r'[^\w\s]',' ',proy) for proy in documentos]
    documentos = [[word for word in texto.lower().split()] for texto in documentos]
    documentos = [remove_noChar(text) for text in documentos]
    #documentos = [remove_tilde(text) for text in documentos]
    documentos = [remove_sw(words_lst, all_stopwords) for words_lst in documentos]
    documentos = [remove_shortW(text) for text in documentos]
    return [' '.join(item) for item in documentos]


In [ ]:
nltk.download('words')

In [8]:
# archivo stopwords
with open(scriptPath + '/stop_words_spanish.txt', 'rb') as f:
    sw_spanish = f.read().decode('latin-1').replace(u'\r', u'').split(u'\n')

sw_spanish

#---------------------------------------------------------------------------------

# conjunto de stopwords de nltk
default_stopwords = nltk.corpus.stopwords.words('spanish')

# Combinar ambos conjuntos de stopwords
all_stopwords = list(set(default_stopwords) | set(sw_spanish) )


all_stopwords

['sentid',
 'tengamos',
 'dado',
 'arriba',
 'pudo',
 'algún',
 'deben',
 'podeis',
 'habría',
 'tampoco',
 'nosotros',
 'qeu',
 'allí',
 'estados',
 'estuvieron',
 'habido',
 'lado',
 'considera',
 'detras',
 'intento',
 'ser',
 'sabeis',
 'sus',
 'otros',
 'teníais',
 'sido',
 'estuvieseis',
 'una',
 'dias',
 'estaremos',
 'seis',
 'unas',
 'para',
 'enseguida',
 'fuesen',
 'tras',
 'aún',
 'unos',
 'ti',
 'éramos',
 'este',
 'tuve',
 'emplean',
 'habrán',
 'fuiste',
 'por',
 'se',
 'embargo',
 'mismo',
 'junto',
 'habréis',
 'ahi',
 'hemos',
 'sé',
 'b',
 'todos',
 'repente',
 'usted',
 'sabe',
 'fueseis',
 'cuánto',
 'tienes',
 'ex',
 'intenta',
 'dio',
 'tendrá',
 'tened',
 'mejor',
 'hubiste',
 'buena',
 'anterior',
 'muchos',
 'haceis',
 'ha',
 'bien',
 'días',
 'vuestros',
 'ya',
 'hayáis',
 'seas',
 'h',
 'estarías',
 'mediante',
 'parece',
 'realizó',
 'ni',
 'hubiese',
 'hicieron',
 'contigo',
 'casi',
 'habré',
 'quizas',
 'les',
 'estuvieran',
 'próximos',
 'proximo',
 'tu

Finalmente, con las funciones de limpieza listas, y la lista de stop words cargada, procedemos a limpiar el texto

### Correr limpieza

In [9]:
# Ejecutemos la limpieza y cronometremos el tiempo que toma correr

t0 = time.time()

clean_text = pd.DataFrame(text_table.Text)

clean_text = clean_text.apply(preProc_docs)

t1 = time.time()

# 16 segundos para procesar todo el texto
total = t1-t0

## Extraer información relevante con expresión regular

In [10]:
#re.search(r'DEPARTAMENTO NACIONAL DE PLANEACIÓN\.(.*?)Departamento Nacional de Planeación', text_table.Text[5])
titles = []
for i in range(1,len(clean_text)):
    try:
        
        temp = re.search('nacional planeación(.*)versión aprobada', clean_text.Text.iloc[i])
        temp2 = temp.group(1).strip()
        temp3 = temp2.replace('CONSEJO NACIONAL DE POLÍTICA ECONÓMICA Y SOCIAL  REPÚBLICA DE COLOMBIA DEPARTAMENTO NACIONAL DE PLANEACIÓN       ','')
        titles.append(temp2)
        
    except:
        pass
    
    #except Exception:
    #    temp = re.search('Documento  CONPES(.*)Versión aprobada', text_table.Text[i])
    
    #if not os.path.isfile(filename):
    #    try: 
    #        urllib.request.urlretrieve(url, filename)
            
    #    except Exception:
    #        pass

len(titles)


#-------------------------------------------------------------------------------
#titles_xlsx = pd.read_excel('C:\\Users\\User\\Desktop\\conpes_list.xlsx')
#titles_2 = pd.DataFrame(titles_xlsx.titulo)
#titles_2_clean = titles_2.apply(preProc_docs).titulo.tolist()

76

In [11]:
titles

['política nacional gestión integral residuos sólidos departamento nacional planeación ministerio ambiente desarrollo sostenible ministerio vivienda ciudad territorio ministerio educación nacional ministerio minas energía comisión regulación agua potable saneamiento básico departamento administrativo nacional estadísticas superintendencia servicios públicos domiciliarios unidad planeación minero energética',
 'concepto favorable nación otorgar garantía fondo empresarial creado ley contratar operaciones pasivas crédito interno suma millones pesos departamento nacional planeación ministerio hacienda crédito público superintendencia servicios públicos domiciliarios',
 'concepto favorable nación contratar empréstitos externos libre destinación rápido desembolso líneas crédito contingentes ocurrencia desastres naturales entidades financieras internacionales organismos multilaterales entidades fomento gobiernos suma usd millones equivalente monedas financiar apropiaciones presupuestales prio

## Almacenar

In [19]:
pickle.dump( titles, open( "titles.p", "wb" ) )